# ND simplex visualisation
#### Danny Dorstijn, 500732510
More than three spacial dimensions is something humans cannot inherently understand, as we are merely three dimensional beings. Still it is a fun excersize to try to understand more about these arbitrary dimensions. A good way of visualizing higher dimensions is by projecting it down. This is a technique already used in 3D visualisation. A virtual camera projects the 3D information onto a 2D plane that is then send to the screen of your computer or tv. We can also use this for projecting higher dimensions into 3D.

In this paper we will look at an implamentation of this projection in higher dimensions. Before we start we first need to import a few libraries. Numpy for matrices calculation, a couple of functions from math and our self-written vector class that handles vector data for any dimensional vector. This class provides basic functionality for linear vector calculation. 

In [1]:
import numpy as np
from math import sqrt, tan

from vector import VectorN

We use simplexes for our visualisation. Simplexes can be calculated for every dimension. A simplex is a generalization of the notion of a triangle or tetrahedron to arbitrary dimensions. And it has specific properties that help us calculate the vertices for this object. The rule for generating a simplex is that you need n+1 vertices.

The coordinates of the vertices of a regular n-dimensional simplex can be obtained from these two properties:
 - For a regular simplex, the distances of its vertices to its center are equal.
 - The dimensional simplex through its center is arccos(−1/n)
 
These can be used as follows. Let vectors (v0, v1, ..., vn) represent the vertices of an n-simplex center the origin, all unit vectors so a distance 1 from the origin, satisfying the first property. The second property means the dot product between any pair of the vectors is − 1 / n. This can be used to calculate positions for them. 

First we create an array of vertices. We return 0 if the dimension is zero. This is just a dot. But for consistency in returning we return a 1-dimensional point. 

In [2]:
dimensions = 4

vertices = []
# Return point if 0th dimension
if dimensions == 0:
    vertices.append(VectorN([0]))

Because we work with unit vectors for distances we can choose a simple starting vector for our calculations. This could be any vector as long as it has a length of 1.

In [3]:
# For the first vertex we choose a point 1, 0, 0, ...
vertices.append(VectorN([1.0] + [0.0]*(dimensions-1)))

The other vertices can be generated from this starting vector by looping through two steps. First we assume for the vector that everything after the (i + 1)th element are zeroes, where i is the . Then we take the dot product of the previous vector with our current and solve the missing element i for the dot product -1 / n.

An exception is the last vector. This extra vector is the opposite of the second to last vector. There is no need for a dot product here. 

Afterwards we calculate the last non-zero element by using the fact that the vector should end up as a unit vector.

Solving the dot product is handled by the solve_dot_product function. 

In [4]:
def solve_dot_product(vertices, index, dimensions):
    dot_product = sum([x**2 for x in vertices[index-1][:index-1]])
    dot_product = (-1.0/dimensions - dot_product) / vertices[index-1][index-1]
    
    return dot_product

And the function for solving the unit vector.

In [5]:
def solve_magnitude(vertices, index):
    return sqrt(1.0 - sum([x**2 for x in vertices[index]]))

This is used by the create_simplex function.

In [6]:
    # Every other vertex can be generated in 2 steps
    for i in range(1, dimensions+1):
        vertices.append(VectorN([0]*dimensions))

        for j in range(i-1):
            vertices[i][j] = vertices[i-1][j]

        # The last vertex is special because it is just the negatve
        # of the vertex before
        if i == dimensions:
            vertices[i][i-1] = -vertices[i-1][i-1]
            break
        # Solve the first missing coordinate
        vertices[i][i-1] = solve_dot_product(vertices, i, dimensions)
        # Solve the second missing coordinate
        vertices[i][i] = solve_magnitude(vertices, i)

For the projection of higher dimensions into three dimension we have a seperate set of functions. Because we do the same projection as from 3d to 2d, we have to create seperate cameras for every dimension. This means we make use of the MVP matrices. MVP is short for Model, View, Projection and these matrices multiplied give the matrix that can project the vertices into a lower dimension. For this example we do not use a translation matrix. The view matrix is renamed to lookat matrix because we create the matrix from two values. The from vector of the camera and the to vector. This is a point the camera is looking at. 

In [7]:
def get_lookat_matrix(dimensions, from_point, to_point):
    """ Return the matrix based on 2 vectors. 
        from_point(cam pos) and to_pont(where you look) """
    # Create a homogeneous matrix from the dimension
    matrix = np.identity(dimensions + 1)
    orthogonal_vectors = []

    for i in range(dimensions - 2):
        orthogonal_vectors.append(VectorN([0.0]*dimensions))
        for j in range(dimensions):
            if (i+1) == j:
                orthogonal_vectors[i][j] = 1.0
        print(orthogonal_vectors[i])
    to_point.subtract(from_point)
    columns = np.identity(dimensions)
    to_point.normalize()
    columns[dimensions-1] = to_point.data

    for i in range(dimensions - 1):
        cross_vectors = []

        for j in range(dimensions -1):
            cross_vectors.append(VectorN([0]*dimensions))

        j = i - (dimensions - 2)
        for c in range(dimensions - 1):
            if j < 0:
                cross_vectors[c] = orthogonal_vectors[(j + (dimensions - 2))]
            elif j == 0:
                cross_vectors[c] = columns[(dimensions - 1)]
            else:
                cross_vectors[c] = columns[(j - 1)]

            j += 1

        columns[i] = VectorN.get_normal(cross_vectors, dimensions).data

        if i != (dimensions - 2):
            np.linalg.norm(columns[i])

    for i in range(dimensions + 1):
        for j in range(dimensions + 1):
            if i < dimensions and j < dimensions:
                matrix[i][j] = columns[j][i]

    return matrix


The code for the calculation of the matrix is quite complicated. First we create n-2 orthagonal vectors. We don't need the first and the last orthogonal vector. Afterwards we get the normal from the from and to points.

After this we create the perpective matrix. This projects the vertices into a frustrum (in 3d).

In [8]:
def get_perspective_matrix(dimensions):
    """ Return the matrix that projects 
        the dimension onto a lower dimension """
    matrix = np.identity(dimensions + 1)
    fov = 1.0 / tan(90.0 / 2.0)

    # Fill the diagonal line with fov apart from the last 2
    for i in range(dimensions + 1):
        for j in range(dimensions+1):
            if i == j and i < dimensions - 1:
                matrix[i][j] = fov

    return matrix

This matrix is very simple. This is just an identity matrix where every one is replaced by have of the angle between the center point of the view and the view angle, apart from the last two 1's.

After we computed these matrices we can multiply them to create our projection matrix. This projection matrix can then be used to project every vertex in the camera.

In [9]:
def get_projection_matrix(dimensions):
    """ returns matrix by multiplying the 
        lookat with the perspective_matrix"""
    from_pos = VectorN([4.0, 4.0, 4, 1.0])
    to_pos = VectorN([0.0, 0.0, 0.0, 0.0])

    lookat_matrix = get_lookat_matrix(dimensions, from_pos, to_pos)
    perspective_matrix = get_perspective_matrix(dimensions)

    return np.matmul(lookat_matrix, perspective_matrix)

In the project3d function we make all the vertices homogeneous. Afterwards multiplying these vertices with the view/projection matrix we reduce the vertices to the n-1th dimensions. We check if the new dimension is equal to three, if not we recursively project it again.

In [10]:
def project3d(vertices, dimensions):
    """ project to lower dimension """
    view_matrix = get_projection_matrix(dimensions)

    for i, item in enumerate(vertices):
        vertices[i].homogeneous()
        np.matmul(vertices[i].data, view_matrix)
        vertices[i].normalize_reduce()
        
    if dimensions - 1 != 3:
        project3d(vertices, dimensions-1)

That's it! Now we can print the list of vertices to see what the outcome is. You can change the dimensions at the top of this article.

In [11]:
for i, item in enumerate(vertices):
    print(item)

[1.0, 0.0, 0.0, 0.0]
[-0.25, 0.9682458365518543, 0, 0]
[-0.25, -0.3227486121839514, 0.9128709291752769, 0]
[-0.25, -0.3227486121839514, -0.45643546458763834, 0.7905694150420949]
[-0.25, -0.3227486121839514, -0.45643546458763834, -0.7905694150420949]
